# Chess.com Data Prep
This notebook generates two necessary CSV files for the Chess.com Titled Tuesday analysis: `final_df.csv`, which contains each match, date, and outcome, and `player_stats.csv`, which contains aggregate player statistics (e.g., how many games they won in each Titled Tuesday).

In [10]:
import os
import pandas as pd

### Match Outcomes

In [11]:
folder_path = '../output/Chess.com/Titled Tuesday Pairings'

df_list = []
len_files = [] 

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        temp_df = pd.read_csv(os.path.join(folder_path, filename))

        temp_df['filename'] = filename.split('.')[0]
        temp_df['date'] = temp_df['filename'].str.extract(r'([a-zA-Z]+-\d{2}-\d{4})')
        
        temp_df['day'] = temp_df['date'].str.extract(r'(\d{2})')
        temp_df['month'] = temp_df['date'].str.extract(r'([a-zA-Z]+)')
        temp_df['year'] = temp_df['date'].str.extract(r'(\d{4})')

        temp_df['white_result'] = temp_df['result'].str.split(' - ').str[0]
        temp_df['black_result'] = temp_df['result'].str.split(' - ').str[1]
        
        temp_df['type'] = temp_df['filename'].str.extract(r'^(early|late)')
        len_files.append(len(temp_df))
        df_list.append(temp_df)

final_df = pd.concat(df_list, ignore_index=True).drop_duplicates()

In [12]:
final_df.shape

(242418, 15)

In [13]:
final_df.head(1)

,white_rank,white_username,white_elo,result,black_rank,black_username,black_elo,filename,date,day,month,year,white_result,black_result,type
0,NM,chess_blitz00,2636,0 - 1,GM,Hikaru,3324,early-titled-tuesday-blitz-april-02-2024-46631...,april-02-2024,02,april,2024,0,1,early


In [14]:
final_df.to_csv('../output/Chess.com/final_df.csv', index=False)

### Player Outcomes

In [15]:
white_wins = final_df[final_df['white_result'] == '1'].groupby('white_username').size() # games won as white
black_wins = final_df[final_df['black_result'] == '1'].groupby('black_username').size() # games won as black

white_games = final_df.groupby('white_username').size() # games played as white
black_games = final_df.groupby('black_username').size() # games played as black

win_counts = pd.concat([white_wins, black_wins], axis=1, keys=['white_wins', 'black_wins']).fillna(0)
win_counts['total_wins'] = win_counts['white_wins'] + win_counts['black_wins'] # total wins as both white and black

total_games = pd.concat([white_games, black_games], axis=1, keys=['white_games', 'black_games']).fillna(0)
total_games['total_games'] = total_games['white_games'] + total_games['black_games'] # total games as both white and black

win_proportion_white = win_counts['white_wins'] / total_games['white_games']
win_proportion_black = win_counts['black_wins'] / total_games['black_games']

stats = pd.concat([
    win_counts['white_wins'], win_counts['black_wins'], 
    total_games['white_games'], total_games['black_games'], 
    win_counts['total_wins'], total_games['total_games'], 
    win_proportion_white.rename('win_proportion_white'), 
    win_proportion_black.rename('win_proportion_black')
], axis=1).fillna(0)

stats['win_proportion'] = stats['total_wins'] / stats['total_games']

stats = stats.reset_index().rename(columns={'index': 'username'})
stats.shape

(4963, 10)

In [16]:
stats.head(1)

,username,white_wins,black_wins,white_games,black_games,total_wins,total_games,win_proportion_white,win_proportion_black,win_proportion
0,01Roman,20.0,14.0,42.0,41.0,34.0,83.0,0.47619,0.341463,0.409639


In [17]:
stats.to_csv('../output/Chess.com/player_stats.csv', index=False)